In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

# Import API key
from config1 import gmap_key

# Configure gmaps API key
gmaps.configure(api_key=gmap_key)

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Qaanaaq,GL,77.4840,-69.3632,29.03,93,100,4.25,overcast clouds
1,1,Thompson,CA,55.7435,-97.8558,57.36,47,20,9.22,few clouds
2,2,Busselton,AU,-33.6500,115.3333,56.08,64,100,13.76,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,29.86,93,90,64.45,overcast clouds
4,4,East London,ZA,-33.0153,27.9116,55.80,67,100,9.75,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Thompson,CA,55.7435,-97.8558,57.36,47,20,9.22,few clouds
2,2,Busselton,AU,-33.6500,115.3333,56.08,64,100,13.76,overcast clouds
4,4,East London,ZA,-33.0153,27.9116,55.80,67,100,9.75,overcast clouds
6,6,Georgetown,MY,5.4112,100.3354,85.93,79,20,5.75,few clouds
7,7,Rikitea,PF,-23.1203,-134.9692,75.15,70,64,12.01,broken clouds
8,8,Butaritari,KI,3.0707,172.7902,82.96,78,24,13.04,few clouds
9,9,Mataura,NZ,-46.1927,168.8643,49.48,98,100,4.90,overcast clouds
10,10,Kapaa,US,22.0752,-159.3190,84.52,63,75,3.00,broken clouds
11,11,Ngorongoro,TZ,-3.2397,35.4875,49.01,95,99,5.57,overcast clouds
12,12,Cherskiy,RU,68.7500,161.3000,63.88,28,0,2.89,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull().sum()
preferred_cities_df.count()

City_ID        643
City           643
Country        638
Lat            643
Lng            643
Max Temp       643
Humidity       643
Cloudiness     643
Wind Speed     643
Description    643
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID        638
City           638
Country        638
Lat            638
Lng            638
Max Temp       638
Humidity       638
Cloudiness     638
Wind Speed     638
Description    638
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Thompson,CA,57.36,few clouds,55.7435,-97.8558,
2,Busselton,AU,56.08,overcast clouds,-33.6500,115.3333,
4,East London,ZA,55.80,overcast clouds,-33.0153,27.9116,
6,Georgetown,MY,85.93,few clouds,5.4112,100.3354,
7,Rikitea,PF,75.15,broken clouds,-23.1203,-134.9692,
8,Butaritari,KI,82.96,few clouds,3.0707,172.7902,
9,Mataura,NZ,49.48,overcast clouds,-46.1927,168.8643,
10,Kapaa,US,84.52,broken clouds,22.0752,-159.3190,
11,Ngorongoro,TZ,49.01,overcast clouds,-3.2397,35.4875,
12,Cherskiy,RU,63.88,clear sky,68.7500,161.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,
          "type": "lodging",
          "key": gmap_key}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print('Hotel not found...skipping')

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
H

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
H

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df.head(25)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,East London,ZA,55.80,overcast clouds,-33.0153,27.9116,Tu Casa
6,Georgetown,MY,85.93,few clouds,5.4112,100.3354,Cititel Penang
7,Rikitea,PF,75.15,broken clouds,-23.1203,-134.9692,Pension Maro'i
9,Mataura,NZ,49.48,overcast clouds,-46.1927,168.8643,Ellie's Villa
11,Ngorongoro,TZ,49.01,overcast clouds,-3.2397,35.4875,Simba Special Campsite B
14,Tottori,JP,71.60,clear sky,35.5000,134.2333,Hotel Monarque Tottori
17,Torbay,CA,49.48,broken clouds,47.6666,-52.7314,Gallows Cove
18,Naze,JP,77.92,overcast clouds,28.3667,129.4833,奄美ポートタワーホテル
21,Hirara,JP,79.07,broken clouds,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
28,Peru,US,63.12,clear sky,40.7537,-86.0689,Cole Porter Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
map_fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
map_fig.add_layer(marker_layer)
# 11b. Display the figure
map_fig

Figure(layout=FigureLayout(height='420px'))